In [22]:
import os
import pandas as pd
from binance.client import Client
from dotenv import load_dotenv

# Load API keys from your .env file
load_dotenv()
API_KEY = os.environ.get("BINANCE_API_KEY")
API_SECRET = os.environ.get("BINANCE_API_SECRET")

In [23]:
client = Client(API_KEY, API_SECRET)
symbols = ['BTCUSDT', 'ETHUSDT', 'BNBUSDT']

In [8]:
def fetch_trades_for_symbol(symbol):
    """
    Fetches all trades for the given symbol.
    Binance returns up to 500 trades per request. For accounts with a long trade history,
    this function paginates using the fromId parameter.
    """
    trades = []
    try:
        last_trade_id = None
        while True:
            if last_trade_id is None:
                trade_batch = client.get_my_trades(symbol=symbol)
            else:
                trade_batch = client.get_my_trades(symbol=symbol, fromId=last_trade_id)
            if not trade_batch:
                break
            trades.extend(trade_batch)
            # If the batch is smaller than the max (500), we assume we've reached the end.
            if len(trade_batch) < 500:
                break
            # Prepare for the next batch.
            last_trade_id = trade_batch[-1]['id'] + 1
    except Exception as e:
        print(f"Error fetching trades for {symbol}: {e}")
    # Add the symbol to each trade dictionary
    for trade in trades:
        trade['symbol'] = symbol
    return trades

In [9]:
all_trades = []
for symbol in symbols:
    print(f"Fetching trades for {symbol}...")
    symbol_trades = fetch_trades_for_symbol(symbol)
    print(f"Found {len(symbol_trades)} trades for {symbol}.")
    all_trades.extend(symbol_trades)

# Create a DataFrame from the collected trades.
df_trades = pd.DataFrame(all_trades)
print("First few trades fetched:")
print(df_trades.head())

Fetching trades for BTCUSDT...
Found 2 trades for BTCUSDT.
Fetching trades for ETHUSDT...
Found 5 trades for ETHUSDT.
Fetching trades for BNBUSDT...
Found 0 trades for BNBUSDT.
First few trades fetched:
    symbol          id      orderId  orderListId           price         qty  \
0  BTCUSDT  3703874115  28970219379           -1  68218.00000000  0.00010000   
1  BTCUSDT  3703874116  28970219379           -1  68218.00000000  0.03035000   
2  ETHUSDT  1376774058  16944035283           -1   3527.80000000  0.10270000   
3  ETHUSDT  1380198694  16993580485           -1   3271.06000000  0.29690000   
4  ETHUSDT  1399380080  17303017615           -1   3075.52000000  0.25840000   

        quoteQty  commission commissionAsset           time  isBuyer  isMaker  \
0     6.82180000  0.00000010             BTC  1722074561017     True    False   
1  2070.41630000  0.00003035             BTC  1722074561017     True    False   
2   362.30506000  0.00010270             ETH  1711821143899     True    F

In [10]:
df_trades

,symbol,id,orderId,orderListId,price,qty,quoteQty,commission,commissionAsset,time,isBuyer,isMaker,isBestMatch
0,BTCUSDT,3703874115,28970219379,-1,68218.00000000,0.00010000,6.82180000,0.00000010,BTC,1722074561017,True,False,True
1,BTCUSDT,3703874116,28970219379,-1,68218.00000000,0.03035000,2070.41630000,0.00003035,BTC,1722074561017,True,False,True
2,ETHUSDT,1376774058,16944035283,-1,3527.80000000,0.10270000,362.30506000,0.00010270,ETH,1711821143899,True,False,True
3,ETHUSDT,1380198694,16993580485,-1,3271.06000000,0.29690000,971.17771400,0.00029690,ETH,1712090997163,True,True,True
4,ETHUSDT,1399380080,17303017615,-1,3075.52000000,0.25840000,794.71436800,0.00025840,ETH,1713481997789,True,False,True
5,ETHUSDT,1545983063,19724707691,-1,2439.65000000,0.20650000,503.78772500,0.00020650,ETH,1724798051199,True,False,True
6,ETHUSDT,1863536205,22247146252,-1,3724.18000000,0.19270000,717.64948600,0.71764949,USDT,1733093022669,False,False,True


In [15]:
###
# client.get_spot_delist_schedule()
# 
# 
# ###

deposit_history = client.get_deposit_history()

In [16]:
df_deposits = pd.DataFrame(deposit_history)
print("Deposit History:")
print(df_deposits.head())

# If you want to calculate the total deposits per currency, you could do something like:
if not df_deposits.empty:
    df_deposits['amount'] = pd.to_numeric(df_deposits['amount'], errors='coerce')
    total_by_currency = df_deposits.groupby('coin')['amount'].sum()
    print("\nTotal Deposited by Currency:")
    print(total_by_currency)

Deposit History:
Empty DataFrame
Columns: []
Index: []


In [29]:
client.get_my_trades(symbol='BTCEUR')

[{'symbol': 'BTCEUR',
  'id': 137684727,
  'orderId': 3918178484,
  'orderListId': -1,
  'price': '61681.48000000',
  'qty': '0.01717000',
  'quoteQty': '1059.07101160',
  'commission': '0.00001717',
  'commissionAsset': 'BTC',
  'time': 1729688945060,
  'isBuyer': True,
  'isMaker': False,
  'isBestMatch': True},
 {'symbol': 'BTCEUR',
  'id': 138093390,
  'orderId': 3948109981,
  'orderListId': -1,
  'price': '63948.87000000',
  'qty': '0.01140000',
  'quoteQty': '729.01711800',
  'commission': '0.00001140',
  'commissionAsset': 'BTC',
  'time': 1730820257476,
  'isBuyer': True,
  'isMaker': False,
  'isBestMatch': True},
 {'symbol': 'BTCEUR',
  'id': 143204249,
  'orderId': 4228924514,
  'orderListId': -1,
  'price': '100972.06000000',
  'qty': '0.00027000',
  'quoteQty': '27.26245620',
  'commission': '0.00000027',
  'commissionAsset': 'BTC',
  'time': 1737730597398,
  'isBuyer': True,
  'isMaker': False,
  'isBestMatch': True},
 {'symbol': 'BTCEUR',
  'id': 143204250,
  'orderId': 

In [18]:
from datetime import datetime
import time

currTime = round((time.time()*1000))
startTime = "01/01/2021"
beginTime = round(datetime.strptime(startTime, "%d/%m/%Y").timestamp()*1000)
params = {
      "beginTime": beginTime,
      "endTime": currTime
         }
# 0 = deposit, 1 = withdrawals
# if no beginTime and endTime specified will give last 30 days

client.fiat_order_history(0, **params)

AttributeError: 'Client' object has no attribute 'fiat_order_history'